In [8]:
# Import necessary libraries
from google.colab import drive
import tarfile
import os
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path to the dataset archive on Google Drive
archive_path = '/content/drive/MyDrive/BreaKHis_v1.tar.gz'

In [4]:
# Directory where the dataset will be extracted
extract_dir = '/content/BreaKHis_dataset'

In [5]:
# Create a directory if it does not exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

In [6]:
# Extract the dataset
with tarfile.open(archive_path, 'r:gz') as archive:
    archive.extractall(path=extract_dir)

print('Extraction completed!')

Extraction completed!


In [7]:
# Path to the dataset directory
base_dir = '/content/BreaKHis_dataset/BreaKHis_v1/histology_slides/breast'

In [9]:
# Prepare data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # using 20% of the data for validation
)

train_it = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    class_mode='binary',  # or 'categorical' if multi-class
    batch_size=32,
    subset='training'
)

val_it = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    class_mode='binary',  # or 'categorical' if multi-class
    batch_size=32,
    subset='validation'
)

Found 6328 images belonging to 2 classes.
Found 1581 images belonging to 2 classes.


In [10]:
# CNN Model
def build_cnn_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # or 'softmax' for multi-class
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Train models
def train_model(model, train_it, val_it):
    history = model.fit(
        train_it,
        steps_per_epoch=train_it.samples // train_it.batch_size,
        validation_data=val_it,
        validation_steps=val_it.samples // val_it.batch_size,
        epochs=5
    )
    return history

In [14]:
cnn_history = train_model(cnn_model, train_it, val_it)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


197/197 ━━━━━━━━━━━━━━━━━━━━ 111s 512ms/step - accuracy: 0.7299 - loss: 0.7937 - val_accuracy: 0.7997 - val_loss: 0.4623
Epoch 2/5
  1/197 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8438 - loss: 0.4779

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8438 - loss: 0.4779 - val_accuracy: 0.7692 - val_loss: 0.4776
Epoch 3/5
197/197 ━━━━━━━━━━━━━━━━━━━━ 96s 474ms/step - accuracy: 0.7991 - loss: 0.4798 - val_accuracy: 0.7915 - val_loss: 0.4645
Epoch 4/5
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.8438 - loss: 0.4345 - val_accuracy: 0.7692 - val_loss: 0.3983
Epoch 5/5
197/197 ━━━━━━━━━━━━━━━━━━━━ 96s 474ms/step - accuracy: 0.8151 - loss: 0.4459 - val_accuracy: 0.8335 - val_loss: 0.3866
